In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
import random
from multiprocessing import Pool
seed_list=list(range(1,1000,100))

In [3]:
total_compounds_list=pkl.load( open("/data/dharp/compounding/datasets/total_compounds_list.pkl",'rb'))
novel_compounds_list = pkl.load( open( "/data/dharp/compounding/datasets/novel_compounds_list.pkl", "rb" ) )
modifiers, heads = zip(*novel_compounds_list)
heads_list=list(set(heads))
modifiers_list=list(set(modifiers))

## Creation of Negative Classes - CorruptHead and CorruptModifier


In [4]:
def corrupt_dataset_creator(choice):
    random.seed(choice)
    corrupt_modifier_list=[]
    corrupt_head_list=[]
    for cur_num,(modifier,head) in enumerate(novel_compounds_list):
    # CorruptHead , Modifier is same
    #print(modifier,head)

        while True:
            temp_corrupt_head=random.choice(heads_list)
            if (modifier,temp_corrupt_head) not in novel_compounds_list:
                if (modifier,temp_corrupt_head) not in total_compounds_list:
                    corrupt_head_list.append((modifier,temp_corrupt_head))
                    break
    # CorruptModifier , Head is same
        while True:
            temp_corrupt_modifier=random.choice(modifiers_list)
            if (temp_corrupt_modifier,head) not in novel_compounds_list:
                if (temp_corrupt_modifier,head) not in total_compounds_list:
                    corrupt_modifier_list.append((temp_corrupt_modifier,head))
                    break
    return corrupt_head_list,corrupt_modifier_list

In [5]:
pool = Pool(10)
results=pool.map(corrupt_dataset_creator,seed_list)

In [7]:
len(results)

10

In [21]:
for num,(corrupt_heads,corrupt_modifiers) in enumerate(results):
    print(num+1)
    pkl.dump( corrupt_heads, open( '/data/dharp/compounding/datasets/corrupt_heads_'+str(num+1)+'.pkl', "wb" ) )
    
    pkl.dump( corrupt_modifiers, open( '/data/dharp/compounding/datasets/corrupt_modifiers_'+str(num+1)+'.pkl', "wb" ) )

1
2
3
4
5
6
7
8
9
10


In [16]:
results[1][1][:20]

[('leukemia_n', 'peaceful_n'),
 ('wartime_n', 'robinson_n'),
 ('fiduciary_n', 'supply_n'),
 ('polymerization_n', 'peaceful_n'),
 ('tasmanian_n', 'standard_n'),
 ('affective_n', 'award_n'),
 ('yards_n', 'joint_n'),
 ('daubert_n', 'companion_n'),
 ('wages_n', 'foundation_n'),
 ('archetype_n', 'gentleman_n'),
 ('weekly_n', 'boat_n'),
 ('minoan_n', 'pay_n'),
 ('parenchymal_n', 'wife_n'),
 ('vegetarian_n', 'othello_n'),
 ('setup_n', 'sage_n'),
 ('side_n', 'robinson_n'),
 ('nothing_n', 'evidence_n'),
 ('smoke_n', 'association_n'),
 ('computer_n', 'crane_n'),
 ('shifts_n', 'chase_n')]

In [17]:
results[1][0][:20]

[('a_n', 'taxonomic_n'),
 ('a_n', 'merged_n'),
 ('a_n', 'o_n'),
 ('india_n', 'sweat_n'),
 ('india_n', 'router_n'),
 ('india_n', 'pan_n'),
 ('india_n', 'instance_n'),
 ('india_n', 'tarzan_n'),
 ('india_n', 'yell_n'),
 ('india_n', 'pulse_n'),
 ('india_n', 'prescribing_n'),
 ('india_n', 'inn_n'),
 ('india_n', 'externality_n'),
 ('india_n', 'rawls_n'),
 ('india_n', 'challenge_n'),
 ('the_n', 'billy_n'),
 ('the_n', 'morality_n'),
 ('the_n', 'terzky_n'),
 ('the_n', 'debacle_n'),
 ('the_n', 'ka_n')]